In [1]:
from tqdm.notebook import tqdm
from pprint import pprint
from dns.resolver import *
import dns.rdatatype
import time
import geoip2.database
import json
import whois

# 加载 GeoLite2-ASN 数据库
reader = geoip2.database.Reader('../GeoLite2-ASN.mmdb')

def get_as_info(ip_address):
    try:
        # 查询 IP 地址的 AS 信息
        response = reader.asn(ip_address)
        return response.autonomous_system_number
    except geoip2.errors.AddressNotFoundError:
        # IP 地址不在数据库中
        return None


resolver_in_use = ["223.5.5.5", "9.9.9.9"]
# use specified resolvers for this task.
new_resolver = Resolver()
new_resolver.nameservers = resolver_in_use

获取所有域名的A记录，以及AS信息，以及whois中的注册商信息

In [2]:
tars = ['edu','gov','other']
domains = {}
for tar in tars:
    with open("../data/%s.domain_ns_info.txt"%tar,'r') as inf:
        for line in inf.readlines():
            d = line.strip().lower().split('\t')[0]
            if d != '~NO~NS~':
                domains[d.strip()] = {"A":[], "AS":[], "REGISTER":""}
    for domain in tqdm(domains,desc="get %s domains' A/AS/REGISTER record"%tar):
        try:
            a_s = new_resolver.resolve(domain, rdtype=dns.rdatatype.RdataType.A).rrset.items
            for a in a_s:
                a = str(a)
                domains[domain]['A'].append(a)
                domains[domain]['AS'].append(get_as_info(a))
            domains[domain]['A'] = list(set(domains[domain]['A']))
            domains[domain]['AS'] = list(set(domains[domain]['AS']))
        except Exception as e:
            print("%s has no a"%domain)
        # 执行WHOIS查询
        try:
            result = whois.whois(domain)
            registrar = result['registrar']
            domains[domain]['REGISTER'] = registrar
        except Exception as e:
            print(e, domain)
# 将字典对象转换为 JSON 格式
json_data = json.dumps(domains, indent=4)

# 将 JSON 数据写入文件
filename = '../data/domains_a_as_register.json'
with open(filename, 'w') as file:
    file.write(json_data)

get edu domains' A/AS/REGISTER record:   0%|          | 0/566 [00:00<?, ?it/s]

edu.cn has no a
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
net has no a
gtld-servers.net has no a
nstld.com has no a
com has no a
cn has no a
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
tsinghua.edu.cn has no a
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
moe.edu.cn has no a
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
neu6.edu.cn has no a
Error trying to connect to socket:

get gov domains' A/AS/REGISTER record:   0%|          | 0/979 [00:00<?, ?it/s]

Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
edu.cn has no a
net has no a
gtld-servers.net has no a
nstld.com has no a
com has no a
Error trying to connect to socket: closing socket - [Errno -3] Temporary failure in name resolution
cn has no a
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
tsinghua.edu.cn has no a
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
Error trying to connect to socket: closing socket - [Errno 104] 

get other domains' A/AS/REGISTER record:   0%|          | 0/3050 [00:00<?, ?it/s]

Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
edu.cn has no a
net has no a
gtld-servers.net has no a
nstld.com has no a
com has no a
cn has no a
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
tsinghua.edu.cn has no a
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
Error trying to connect to socket: closing socket - [Errno 104] Connection 